In [1]:
# U-Net model
# coded by st.watermelon
import glob
import shutil
import os
import random
from PIL import Image
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split,StratifiedKFold
from tensorflow import keras
import numpy as np
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras import layers
import tensorflow as tf
import PIL
import matplotlib.pyplot as plt
from PIL import ImageOps
os.environ["CUDA_VISIBLE_DEVICES"] = "4"

2023-07-19 19:05:23.030430: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-19 19:05:23.768075: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [7]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Reshape, Permute, Activation, Input, \
    add, multiply
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.layers import Lambda
import tensorflow.keras as K

def up_and_concate(down_layer, layer, data_format='channels_first'):
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: layers.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: layers.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])

    return concate


def attention_up_and_concate(down_layer, layer, data_format='channels_first'):
    if data_format == 'channels_first':
        in_channel = down_layer.get_shape().as_list()[1]
    else:
        in_channel = down_layer.get_shape().as_list()[3]

    # up = Conv2DTranspose(out_channel, [2, 2], strides=[2, 2])(down_layer)
    up = UpSampling2D(size=(2, 2), data_format=data_format)(down_layer)

    layer = attention_block_2d(x=layer, g=up, inter_channel=in_channel // 4, data_format=data_format)

    if data_format == 'channels_first':
        my_concat = Lambda(lambda x: layers.concatenate([x[0], x[1]], axis=1))
    else:
        my_concat = Lambda(lambda x: layers.concatenate([x[0], x[1]], axis=3))

    concate = my_concat([up, layer])
    return concate


def attention_block_2d(x, g, inter_channel, data_format='channels_first'):
    # theta_x(?,g_height,g_width,inter_channel)

    theta_x = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(x)

    # phi_g(?,g_height,g_width,inter_channel)

    phi_g = Conv2D(inter_channel, [1, 1], strides=[1, 1], data_format=data_format)(g)

    # f(?,g_height,g_width,inter_channel)

    f = Activation('relu')(add([theta_x, phi_g]))

    # psi_f(?,g_height,g_width,1)

    psi_f = Conv2D(1, [1, 1], strides=[1, 1], data_format=data_format)(f)

    rate = Activation('sigmoid')(psi_f)

    # rate(?,x_height,x_width)

    # att_x(?,x_height,x_width,x_channel)

    att_x = multiply([x, rate])

    return att_x


def res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

              padding='same', data_format='channels_first'):
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    layer = input_layer
    for i in range(2):
        layer = Conv2D(out_n_filters // 4, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)
        if batch_normalization:
            layer = BatchNormalization()(layer)
        layer = Activation('relu')(layer)
        layer = Conv2D(out_n_filters // 4, kernel_size, strides=stride, padding=padding, data_format=data_format)(layer)
        layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(layer)

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer
    out_layer = add([layer, skip_layer])
    return out_layer


# Recurrent Residual Convolutional Neural Network based on U-Net (R2U-Net)
def rec_res_block(input_layer, out_n_filters, batch_normalization=False, kernel_size=[3, 3], stride=[1, 1],

                  padding='same', data_format='channels_first'):
    if data_format == 'channels_first':
        input_n_filters = input_layer.get_shape().as_list()[1]
    else:
        input_n_filters = input_layer.get_shape().as_list()[3]

    if out_n_filters != input_n_filters:
        skip_layer = Conv2D(out_n_filters, [1, 1], strides=stride, padding=padding, data_format=data_format)(
            input_layer)
    else:
        skip_layer = input_layer

    layer = skip_layer
    for j in range(2):

        for i in range(2):
            if i == 0:

                layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                    layer)
                if batch_normalization:
                    layer1 = BatchNormalization()(layer1)
                layer1 = Activation('relu')(layer1)
            layer1 = Conv2D(out_n_filters, kernel_size, strides=stride, padding=padding, data_format=data_format)(
                add([layer1, layer]))
            if batch_normalization:
                layer1 = BatchNormalization()(layer1)
            layer1 = Activation('relu')(layer1)
        layer = layer1

    out_layer = add([layer, skip_layer])
    return out_layer
smooth = 1.
def dice_coef(y_true, y_pred):
    flatten_layer = K.layers.Flatten()
    y_true_f = flatten_layer(y_true)
    y_pred_f = flatten_layer(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)
########################################################################################################
# Define the neural network
def unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input(( img_w, img_h,3))
    x = inputs
    depth = 4
    features = 64
    skips = []
    for i in range(depth):
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format= data_format)(x)
        features = features * 2

    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    for i in reversed(range(depth)):
        features = features // 2
        # attention_up_and_concate(x,[skips[i])
        x = UpSampling2D(size=(2, 2), data_format=data_format)(x)
        x = concatenate([skips[i], x], axis=1)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv6 = Conv2D(n_label, (1, 1), padding='same', activation='sigmoid',data_format=data_format)(x)
    model = Model(inputs=inputs, outputs=conv6)

    model.compile(optimizer=Adam(lr=1e-5), loss=[dice_coef_loss], metrics=['accuracy', dice_coef])
    return model


########################################################################################################
#Attention U-Net
def att_unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input(( img_w, img_h,3))
    x = inputs
    depth = 4
    features = 64
    skips = []
    for i in range(depth):
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format='channels_first')(x)
        features = features * 2

    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
    x = Dropout(0.2)(x)
    x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    for i in reversed(range(depth)):
        features = features // 2
        x = attention_up_and_concate(x, skips[i], data_format=data_format)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)
        x = Dropout(0.2)(x)
        x = Conv2D(features, (3, 3), activation='relu', padding='same', data_format=data_format)(x)

    conv6 = Conv2D(n_label, (1, 1), padding='same', activation='sigmoid',data_format=data_format)(x)
    model = Model(inputs=inputs, outputs=conv6)

    model.compile(optimizer=Adam(lr=1e-5), loss=[dice_coef_loss], metrics=['accuracy', dice_coef])
    return model


########################################################################################################
#Recurrent Residual Convolutional Neural Network based on U-Net (R2U-Net)
def r2_unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input(( img_w, img_h,3))
    x = inputs
    depth = 4
    features = 20
    skips = []
    for i in range(depth):
        x = rec_res_block(x, features, data_format=data_format)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format=data_format)(x)

        features = features * 2

    x = rec_res_block(x, features, data_format=data_format)

    for i in reversed(range(depth)):
        features = features // 2
        x = up_and_concate(x, skips[i], data_format=data_format)
        x = rec_res_block(x, features, data_format=data_format)

    conv6 = Conv2D(n_label, (1, 1), padding='same', activation='sigmoid',data_format=data_format)(x)
    model = Model(inputs=inputs, outputs=conv6)
    model.compile(optimizer=Adam(lr=1e-6), loss=[dice_coef_loss], metrics=['accuracy', dice_coef])
    return model

########################################################################################################
#Attention R2U-Net
def att_r2_unet(img_w, img_h, n_label, data_format='channels_first'):
    inputs = Input(( img_w, img_h,3))
    x = inputs
    depth = 4
    features = 32
    skips = []
    for i in range(depth):
        x = rec_res_block(x, features, data_format=data_format)
        skips.append(x)
        x = MaxPooling2D((2, 2), data_format=data_format)(x)

        features = features * 2

    x = rec_res_block(x, features, data_format=data_format)

    for i in reversed(range(depth)):
        features = features // 2
        x = attention_up_and_concate(x, skips[i], data_format=data_format)
        x = rec_res_block(x, features, data_format=data_format)

    conv6 = Conv2D(n_label, (1, 1), padding='same', activation="sigmoid",data_format=data_format)(x)
   
    model = Model(inputs=inputs, outputs=conv6)
    model.compile(optimizer=Adam(lr=1e-6), loss=[dice_coef_loss], metrics=['accuracy', dice_coef])
    return model

In [3]:
mask_filenames = glob.glob('../../data/CycleGANData/wholeslide/image/*.tiff')
mask_filenames=[f.replace('/image', '/mask') for f in mask_filenames]
mask_df = pd.DataFrame()
mask_df['filename'] = mask_filenames
mask_df['mask_percentage'] = 0
mask_df['labels'] = 0
mask_df.set_index('filename', inplace=True)
for file in mask_filenames:
    mask_df.loc[file, 'mask_percentage'] = np.array(Image.open(file)).sum()/(512*512*255)
mask_df.loc[mask_df.mask_percentage > 0.01, 'labels'] = 1

In [4]:
train_valid_filenames = glob.glob('../../data/CycleGANData/wholeslide/image/*.tiff')

train_filenames, valid_filenames = train_test_split(train_valid_filenames,test_size = 0.2,train_size=0.8, random_state = 10)
mask_train_filenames = [f.replace('/image', '/mask') for f in train_filenames]
mask_valid_filenames = [f.replace('/image', '/mask') for f in valid_filenames]

In [5]:
train_x = np.zeros((len(train_filenames),512,512,3))
valid_x = np.zeros((len(valid_filenames),512,512,3))
train_mask_y = np.zeros((len(train_filenames),512,512))
valid_mask_y = np.zeros((len(valid_filenames),512,512))
train_y = np.zeros((len(train_filenames)))
valid_y = np.zeros((len(valid_filenames)))
for (index, image) in enumerate(train_filenames[:]):
    train_x[index] = (255-np.array(Image.open(image)))/255
    
# 검증용  및 변병여부 
for (index, image) in enumerate(valid_filenames[:]):
    valid_x[index] = (255-np.array(Image.open(image)))/255
    
# 훈련용 마스크 답지  및 병변여부 
for (index, image) in enumerate(mask_train_filenames[:]):
    train_mask_y[index] = np.array(Image.open(image))/255
    train_y[index] = mask_df.loc[image, 'labels']
    
# 검증용 병변 답지  및 병변여부 
for (index, image) in enumerate(mask_valid_filenames[:]):
    valid_mask_y[index] = np.array(Image.open(image))/255
    valid_y[index] = mask_df.loc[image, 'labels']

In [9]:
model = att_r2_unet(512,512,1,data_format='channels_last')
model.compile(optimizer=keras.optimizers.Adam(lr=1e-4), loss=dice_coef_loss, metrics=dice_coef)
callbacks = [
        keras.callbacks.ModelCheckpoint("../../model/Tumor_segmentation_Attention_{epoch:03d}.h5", monitor='val_loss'),
    ]
history=model.fit(train_x, [train_mask_y], validation_data=(valid_x, [valid_mask_y]),callbacks=callbacks,epochs = 300, batch_size =4)

Epoch 1/300
227/227 [==============================] - 56s 169ms/step - loss: 0.8540 - dice_coef: 0.1460 - val_loss: 0.8532 - val_dice_coef: 0.1464
Epoch 2/300
227/227 [==============================] - 37s 161ms/step - loss: 0.8540 - dice_coef: 0.1462 - val_loss: 0.8532 - val_dice_coef: 0.1464
Epoch 3/300
227/227 [==============================] - 37s 162ms/step - loss: 0.8541 - dice_coef: 0.1458 - val_loss: 0.8532 - val_dice_coef: 0.1464
Epoch 4/300
227/227 [==============================] - 37s 163ms/step - loss: 0.8543 - dice_coef: 0.1459 - val_loss: 0.8532 - val_dice_coef: 0.1464
Epoch 5/300
227/227 [==============================] - 37s 164ms/step - loss: 0.8545 - dice_coef: 0.1453 - val_loss: 0.8532 - val_dice_coef: 0.1464
Epoch 6/300
227/227 [==============================] - 37s 162ms/step - loss: 0.8539 - dice_coef: 0.1460 - val_loss: 0.8532 - val_dice_coef: 0.1464
Epoch 7/300
227/227 [==============================] - 39s 173ms/step - loss: 0.8541 - dice_coef: 0.1461 - val_l